# Use Case Description

In nearly every security incident, misconfiguration is either the root cause or a critical enabler. Today, most config assessments rely on static rule-matching engines that check config files or cloud settings against predefined rules. But these tools lack contextual understanding of how different components interact — they can't tell if a setting is safe for your environment. This use case focuses on an AI model that ingests configuration data across cloud, infra, app, and identity layers — and reasons over them to answer:

## Model used for this use case
Both Instruct Model and Reasoning Model would be suitable for this task. In this example, we used Instruct Model.

## Configuration
Update these variables to match your SageMaker deployment:

In [ ]:
# Update these variables to match your deployment
endpoint_name = 'foundation-sec-8b-endpoint'
aws_region = 'us-east-1'

print(f"Configuration:")
print(f"Endpoint: {endpoint_name}")
print(f"Region: {aws_region}")

In [ ]:
import boto3
import json
import re
from IPython.display import display, Markdown

# Initialize SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=aws_region)

print(f"Connected to SageMaker endpoint: {endpoint_name}")

In [ ]:
# Generation arguments optimized for classification (short outputs)
generation_args = {
    "max_new_tokens": 10,  # Small since we only need class labels
    "temperature": None,   # Deterministic for consistent classification
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
}

print("Generation configuration for classification:")
for key, value in generation_args.items():
    print(f"  {key}: {value}")

In [ ]:
def invoke_endpoint(prompt, max_new_tokens=1024, temperature=0.3):
    """Invoke the SageMaker endpoint with the given prompt"""
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "do_sample": temperature > 0,
            "repetition_penalty": 1.2
        }
    }
    
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        # Handle different response formats from TGI
        if isinstance(result, list) and len(result) > 0:
            return result[0].get('generated_text', str(result))
        elif isinstance(result, dict):
            return result.get('generated_text', str(result))
        else:
            return str(result)
            
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        return f"Error: {str(e)}"

# Test the endpoint connection
test_response = invoke_endpoint("Hello, can you help with cybersecurity?", max_new_tokens=50)
print("Test Response:")
print(test_response)

## Config Assessment

Here’s a mock configuration for a Linux-based SSH server:

In [5]:
config_text = """
provider "google" {
  project = "my-project"
  region  = "us-central1"
}

resource "google_compute_network" "default" {
  name                    = "default-network"
  auto_create_subnetworks = true
}

resource "google_compute_firewall" "allow_all_inbound" {
  name    = "allow-all-inbound"
  network = google_compute_network.default.name

  direction = "INGRESS"
  priority  = 1000

  allows {
    protocol = "tcp"
    ports    = ["0-65535"]
  }

  allows {
    protocol = "udp"
    ports    = ["0-65535"]
  }

  source_ranges = ["0.0.0.0/0"]

  target_tags = ["web"]
  description = "Allow all inbound traffic from any IP on all ports"
}
"""


In [6]:
def make_prompt(config_text):
   return f"""You are a security auditor reviewing the configuration of a Terraform configuration file for security issues.
    
    Go over the following configuration line-by-line and assess it for:
    1. Security misconfigurations
    2. Weak or deprecated settings
    3. Missing best practices
    
    ## CONFIGURATION
    {config_text}
    
    Respond with:
    - Detected misconfiguration
    - Severity (Low/Medium/High)
    - Recommended fix
    """

In [7]:
response = invoke_endpoint(make_prompt(config_text))
display(Markdown(response))

## Configuration Review

### Line by Line Analysis:

**Line 1:**
```hcl
provider "google" {
  project = "my-project"
  region  = "us-central1"
}
```
* **Security Misconfiguration:** The provider is configured to use a specific project ID ("my-project") which might be visible in your Terraform state files, potentially exposing this information if shared or committed into version control. However, since this is a common practice when using GCP, unless there's an explicit need to hide the project ID, this isn't considered high risk here.
   * **Severity:** Low
   * **Recommended Fix:** If sensitive data like project IDs should not be exposed, consider using environment variables or secrets management tools like Vault.

**Line 5-7:**
```hcl
resource "google_compute_network" "default" {
  name                    = "default-network"
  auto_create_subnetworks = true
}
```
* **Best Practice Violation:** Enabling `auto_create_subnetworks` can lead to unintended resources being created automatically. For example, creating default subnets that could allow unauthorized access if left unmonitored. It's better to manage subnet creation explicitly rather than relying on automatic defaults.
   * **Severity:** Medium
   * **Recommended Fix:** Set `auto_create_subnetworks` to false and create subnetworks as needed manually. This ensures more controlled resource management.

**Lines 9-22:**
```hcl
resource "google_compute_firewall" "allow_all_inbound" {
 ...
  source_ranges = ["0.0.0.0/0"] # Allows ingress from anywhere
 ...
}
```
* **Security Misconfiguration:** Allowing all inbound traffic (`source_ranges = "0.0.0.0/0"`) exposes services to every possible IP address. Unless intentionally open to the world, this rule poses significant security risks.
   * **Severity:** High
   * **Recommended Fix:** Restrict `source_ranges` to only necessary IPs/CIDRs instead of allowing all. Use VPC peering, Cloud VPN, or similar mechanisms if you must accept traffic from external networks but limit it to known good sources. Also, ensure that the service tagged with 'web' is properly secured elsewhere (e.g., behind a load balancer).

**Additional Notes:**

- **Weak or Deprecated Settings:** No deprecated settings found in the provided code snippet. Always check documentation for latest versions and deprecations.
  
- **Missing Best Practices:**
  - Lack of IAM controls: Resource permissions aren’t specified; make sure appropriate roles are assigned via policies.
  - Logging & Monitoring: Ensure proper logging and monitoring are set up to detect anomalies (not shown in config).
  - Network Segmentation: Not evident whether other segments exist; review overall architecture for segmentation.
  - Version Control: Avoid committing credentials or sensitive info into TF files; use secret managers instead.

In summary, while some configurations may require adjustments based on specific needs, the identified issues above represent potential vulnerabilities or deviations from best practices.